In [1]:
from unsloth import FastLanguageModel
import torch
import os
from dotenv import load_dotenv

load_dotenv()

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

huggingface_token = os.getenv("HUGGING_FACE_TOKEN")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.3-70B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=huggingface_token,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ubuntu/us-south-2/hello-fine-tune/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 9.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.5 patched 80 layers with 80 QKV layers, 80 O layers and 80 MLP layers.


In [3]:
import json

resolved_prompts = []
with open('swe-bench-fine-tune/resolved_prompts.jsonl', 'r') as f:
    for line in f:
        resolved_prompts.append(json.loads(line))

resolved_prompts

[{'id': 'marshmallow-code__marshmallow-1164',
  'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an expert software engineer with deep knowledge of code analysis, debugging, and best practices in software development.             You are given the following problem:\n\n<problem_statement>\n2.x: Nested(many=True) eats first element from generator value when dumping\nAs reproduced in Python 3.6.8:\n\n```py\nfrom marshmallow import Schema, fields\n\nclass O(Schema):\n    i = fields.Int()\n\nclass P(Schema):\n    os = fields.Nested(O, many=True)\n\ndef gen():\n    yield {\'i\': 1}\n    yield {\'i\': 0}\n\np = P()\np.dump({\'os\': gen()})\n# MarshalResult(data={\'os\': [{\'i\': 0}]}, errors={})\n```\n\nProblematic code is here:\n\nhttps://github.com/marshmallow-code/marshmallow/blob/2.x-line/src/marshmallow/fields.py#L447\n\nAnd here:\n\nhttps://github.com/marshmallow-code/marshmallow/blob/2.x-line/src/marshmallow/schema.py#L832\n\nThe easiest solution would

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import Dataset

dataset = Dataset.from_list([{"text": item['prompt']} for item in resolved_prompts])

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 20,
        # max_steps = 60,
        num_train_epochs = 10, # For longer training runs!
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/13 [00:00<?, ? examples/s]

Map (num_proc=2): 100%|██████████| 13/13 [00:00<00:00, 13.36 examples/s]
